In [ ]:
import warnings
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['M1','M5','M15','H1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]

    Src['Datetime']                     = Query['datetime']
    Src['Date']                         = Query['datetime'].dt.date
    Src['Time']                         = Query['datetime'].dt.time

    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    Src['Change']       = (Src['Close'] - Src['Open'])
    Src['Hilo']         = (Src['High']  - Src['Low'])

    Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if (x <  0) else nan)
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['T'] = Calc.groupby(['Symbol','TF','Date'], sort=0)['Time'].rank()

    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']

    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']
    return Calc

Calc = CALCULATIONS(Src)
Calc

# Stats

In [15]:
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['Hilo']].describe().round(0).astype(int)

Hilo                                   
     count mean  std min  25%  50%  75%   max
TF                                           
M5   99094  199  127   0  114  167  247  3565
M15  44863  384  246   0  218  324  481  6275
H1   10620  752  483  75  419  635  963  7753

In [16]:
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['HL Pos']].describe().round(0).astype(int)

HL Pos                                   
     count mean  std min  25%  50%  75%   max
TF                                           
M5   51090  196  129   0  112  165  244  3565
M15  22812  379  249   0  214  320  474  6275
H1    5374  740  486  75  415  623  944  7753

In [17]:
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['HL Neg']].describe().round(0).astype(int)

HL Neg                                    
     count mean  std   min  25%  50%  75%  max
TF                                            
M5   48004 -201  125 -2722 -251 -170 -117  -10
M15  22051 -389  242 -3047 -490 -329 -223  -45
H1    5246 -764  479 -5016 -980 -648 -423 -101

In [18]:
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['Change']].describe().round(0).astype(int)

Change                                    
     count mean  std   min  25% 50%  75%   max
TF                                            
M5   99094    0  139 -2670  -67   0   67  3070
M15  44863    0  267 -2962 -127   0  126  5312
H1   10620   -2  531 -4266 -252   0  261  5561

In [19]:
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['Chg Abs']].describe().round(0).astype(int)

Chg Abs                                   
      count mean  std min  25%  50%  75%   max
TF                                            
M5    99094   96  101   0   30   67  128  3070
M15   44863  184  194   0   56  127  247  5312
H1    10620  368  382   0  110  256  493  5561

In [20]:
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['Chg Pos']].describe().round(0).astype(int)

Chg Pos                                   
      count mean  std min  25%  50%  75%   max
TF                                            
M5    51090   93  101   0   28   65  124  3070
M15   22812  181  195   0   53  124  244  5312
H1     5374  362  380   0  108  257  483  5561

In [21]:
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['Chg Neg']].describe().round(0).astype(int)

Chg Neg                                   
      count mean  std   min  25%  50%  75% max
TF                                            
M5    48004  -99  100 -2670 -132  -70  -33  -5
M15   22051 -188  192 -2962 -250 -130  -59  -5
H1     5246 -375  385 -4266 -502 -255 -112  -5

# Snippets

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src['Change']       = (Src['Close'] - Src['Open'])
    Src['Hilo']         = (Src['High']  - Src['Low'])
    Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if (x <  0) else nan)
    return Src 


def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['T'] = Calc.groupby(['Symbol','TF','Date'], sort=0)['Time'].rank()

    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']

    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']
    return Calc


Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['Hilo']].describe().round(0).astype(int)
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['HL Pos']].describe().round(0).astype(int)
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['HL Neg']].describe().round(0).astype(int)

Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['Change']].describe().round(0).astype(int)
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['Chg Abs']].describe().round(0).astype(int)
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['Chg Pos']].describe().round(0).astype(int)
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['Chg Neg']].describe().round(0).astype(int)